<div style='background-color: #fff7f7; border: 3px solid Red;'>
    <font size="+1" color="Red">
        <b>Please comment and upvote if you like this work!</b>
    </font>
</div>
<div style='background-color: #fff7f7; border: 3px solid Blue;'>
    <font size="+1" color="Red">
        <b>Take a look at the other notebooks in my profile ->
        <a href="https://www.kaggle.com/killa92/code">  Kaggle Notebooks </a></b>
    </font>
</div>    
<div style='background-color: #fff7f7; border: 3px solid yellow;'>
    <font size="+1" color="Red">
        <b><span style='color: green;'>Let's Support Each Other!!</span></b>
    </font>
</div>

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>🗂1. Install libraries and Download a pre-trained Models  🗂️</b></font>
</div>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

 <div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>📈2. Prepare Datasets and Dataloaderss 📈</b></font>
</div>

In [ ]:
from glob import glob
import os
import shutil

HOME = "/kaggle/working"
def make_ds(root, path_to_copy):
    
    os.makedirs(path_to_copy, exist_ok = True)
    #os.makedirs(f"{path_to_copy}", exist_ok = True)
    
    files = glob(f"{root}/*")
    for idx, file in enumerate(files):
        #if os.path.isdir(file): os.system(f"cp -r {file} {path_to_copy}")
        if os.path.isdir(file): 
            shutil.rmtree(file)
            shutil.copytree(file,path_to_copy)
        #elif os.path.isfile(file): os.system(f"cp {file} {path_to_copy}")
        elif os.path.isfile(file):  
            os.remove(file)
            shutil.copy(file,path_to_copy)
            
root = "/kaggle/input/drone-detection"
make_ds(root = root, path_to_copy = f"{HOME}/datasets")

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>✨3. AI Model Trainn ✨</b></font>
</div>

In [ ]:
train_results = model.train(
    data=f"{HOME}/datasets/data.yaml",  # path to dataset YAML
    epochs=10,  # number of training epochs
    imgsz=480,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>😍4.. Learning Curves 😍</b></font>
</div>

In [ ]:
from PIL import Image
Image.open(f"runs/detect/train/confusion_matrix.png")

In [ ]:
Image.open(f"runs/detect/train/results.png")

In [ ]:
Image.open(f"runs/detect/train/P_curve.png")

In [ ]:
Image.open(f"runs/detect/train/PR_curve.png")

In [ ]:
Image.open(f"runs/detect/train/val_batch0_pred.jpg")

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>✅6. Inference with Test Set ImagesM ✅</b></font>
</div>

In [ ]:
inference_results = model("datasets/test/images")

In [ ]:
import cv2, numpy as np

for idx, r in enumerate(inference_results):
    or_im_rgb = np.array(Image.open(r[0].path).convert("RGB"))
    if idx == 5: break    
    print(f"Image#{idx + 1}")
    for i in r:
        for bbox in i.boxes:
            box = bbox.xyxy[0]
            x1, y1, x2, y2 = box
            coord1, coord2 = (int(x1), int(y1)), (int(x2), int(y2))
            cv2.rectangle(or_im_rgb, coord1, coord2, color=(255,0,0), thickness=2)
    display(Image.fromarray(or_im_rgb))